In [2]:
import ee
import geopandas as gpd
import pandas as pd
from datetime import datetime

# --------- 1. Initialize Earth Engine
ee.Initialize()


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [6]:
# --------- 2. Load shapefile as GeoDataFrame and convert to ee.Geometry
def load_geometry_from_shapefile(shapefile_path):
    gdf = gpd.read_file(shapefile_path)
    geom = ee.Geometry(gdf.geometry[0].__geo_interface__)
    return geom

# --------- 3. Get list of monthly dates between year range
def get_monthly_date_range(start_year, end_year):
    return [datetime(y, m, 1).strftime('%Y-%m') for y in range(start_year, end_year + 1) for m in range(1, 13)]

# --------- 4. Main function to extract water area time series
def extract_monthly_water_area(shapefile_path, start_year, end_year):
    roi = load_geometry_from_shapefile(shapefile_path)
    date_list = get_monthly_date_range(start_year, end_year)

    # Compute total area in km² once
    total_area_km2 = ee.Image.pixelArea().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=roi,
        scale=30,
        maxPixels=1e10
    ).getInfo()['area'] / 1e6

    # Load JRC dataset
    dataset = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory") \
        .filterBounds(roi) \
        .filterDate(f'{start_year}-01-01', f'{end_year}-12-31')

    def process_image(img):
        date_str = img.date().format('YYYY-MM')
        water_mask = img.select('water').eq(2)
        area_img = water_mask.multiply(ee.Image.pixelArea()).divide(1e6)  # km²

        stats = area_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=roi,
            scale=30,
            maxPixels=1e10
        )

        return ee.Feature(None, {
            'month': date_str,
            'area_km2': stats.get('water'),
            'total_area_km2': total_area_km2
        })

    features = dataset.map(process_image).getInfo()['features']

    df = pd.DataFrame([
        {
            'month': f['properties']['month'],
            'area_km2': f['properties']['area_km2'],
            'total_area_km2': f['properties']['total_area_km2']
        }
        for f in features
    ])

    return df

# Example usage
df = extract_monthly_water_area(
    shapefile_path=r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\StudyAreas\Delta_UCB\Delta_UCB_WGS84.shp",
    start_year=1984,
    end_year=2021
)

# Save or display
print(df)
df.to_csv(r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\Code\Troubleshooting\monthly_water_area.csv", index=False)


       month    area_km2  total_area_km2
0    1984-03    0.000000    23317.064473
1    1984-04    0.000000    23317.064473
2    1984-05    0.000000    23317.064473
3    1984-06  356.916344    23317.064473
4    1984-07  192.742736    23317.064473
..       ...         ...             ...
449  2021-08  113.124581    23317.064473
450  2021-09   98.238668    23317.064473
451  2021-10   63.047501    23317.064473
452  2021-11   54.976423    23317.064473
453  2021-12   48.322448    23317.064473

[454 rows x 3 columns]
